In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from keras.models import Sequential
import tensorflow as tf
import tensorflow_datasets as tfds

from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Convolution2D, MaxPooling2D

from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import random
from numpy import *
from PIL import Image

In [3]:
# Define paths and categories
Spath_test = "/content/drive/MyDrive/Crop___Disease/Potato"
CATEGORIES =["Potato___Early_Blight", "Potato___Healthy", "Potato___Late_Blight"]
IMG_SIZE = 200
NUM_CLASSES = len(CATEGORIES)

In [15]:
# Prepare training data
training = []

def createTrainingData():
    for category in CATEGORIES:
        path = os.path.join(Spath_test, category)
        print(path)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training.append([new_array, class_num])
            except Exception as e:
                print(f"Error loading image {img}: {e}")

createTrainingData()

/content/drive/MyDrive/Crop___Disease/Potato/Potato___Early_Blight
/content/drive/MyDrive/Crop___Disease/Potato/Potato___Healthy
/content/drive/MyDrive/Crop___Disease/Potato/Potato___Late_Blight


In [16]:
# Shuffle and split the data
np.random.shuffle(training)
X, y = [], []

for features, label in training:
    X.append(features)
    y.append(label)

In [17]:
# Convert to NumPy arrays and normalize
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3).astype('float32') / 255.0
y = np.array(y)

In [18]:
# Convert labels to categorical
from tensorflow.keras.utils import to_categorical
Y = to_categorical(y, NUM_CLASSES)

In [19]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=4)

In [20]:
# Model parameters
batch_size = 32
nb_epochs = 5

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 4)

In [37]:
batch_size = 32
nb_classes = 4
nb_epochs = 5
img_rows, img_columns = 200, 200
img_channel = 3
nb_filters = 32
nb_pool = 2
nb_conv = 3

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu,
                           input_shape=(200, 200, 3)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dense(4,  activation="softmax")
])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size = batch_size, epochs = nb_epochs, verbose = 1, validation_data = (X_test, y_test))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 10s 142ms/step - accuracy: 0.5944 - loss: 1.1813 - val_accuracy: 0.6775 - val_loss: 0.7218
Epoch 2/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.8523 - loss: 0.3927 - val_accuracy: 0.9049 - val_loss: 0.2482
Epoch 3/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.9556 - loss: 0.1430 - val_accuracy: 0.9281 - val_loss: 0.2051
Epoch 4/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 0.9676 - loss: 0.0855 - val_accuracy: 0.9327 - val_loss: 0.1724
Epoch 5/5
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - accuracy: 0.9878 - loss: 0.0395 - val_accuracy: 0.9443 - val_loss: 0.1616


In [38]:
# Save the model manually in .h5 format after training
model.save('Potato_model1.h5')  # Save the model in .h5 format
print("Model saved as 'Potato_model1.h5")

Model saved as 'Potato_model1.h5


In [24]:
# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print("Test Loss: ", score[0])
print("Test Accuracy: ", score[1])

Test Loss:  0.13233628869056702
Test Accuracy:  0.9489558935165405


In [25]:
# Plot training & validation accuracy values using Plotly
import plotly.express as px
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(nb_epochs)), y=history.history['accuracy'], mode='lines+markers', name='Train Accuracy'))
fig.add_trace(go.Scatter(x=list(range(nb_epochs)), y=history.history['val_accuracy'], mode='lines+markers', name='Val Accuracy'))
fig.update_layout(title='Model Accuracy', xaxis_title='Epoch', yaxis_title='Accuracy')
fig.show()

In [26]:
# Plot training & validation loss values using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(nb_epochs)), y=history.history['loss'], mode='lines+markers', name='Train Loss'))
fig.add_trace(go.Scatter(x=list(range(nb_epochs)), y=history.history['val_loss'], mode='lines+markers', name='Val Loss'))
fig.update_layout(title='Model Loss', xaxis_title='Epoch', yaxis_title='Loss')
fig.show()

In [27]:
# Predict an image
image = X_test[0]
image = np.expand_dims(image, axis=0)  # Add batch dimension
predictions = model.predict(image)
predicted_class = np.argmax(predictions[0])
print("Predicted class:", CATEGORIES[predicted_class])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
Predicted class: Potato___Early_Blight


In [28]:
def predict_image(image_path, model):
    # Load and preprocess the image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img.astype('float32') / 255.0  # Normalize the image
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    # Make prediction
    predictions = model.predict(img)
    predicted_class = np.argmax(predictions[0])

    # Return the class label
    return CATEGORIES[predicted_class]


In [40]:
image_path = "/content/drive/MyDrive/Crop___Disease/Potato/Potato___Early_Blight/cf761f3d-755a-42b2-83cb-729443246c6a___RS_Early.B 7319.JPG"  # Replace with the path to the user-provided image
predicted_disease = predict_image(image_path, model)
print(f"The predicted disease is: {predicted_disease}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
The predicted disease is: Potato___Early_Blight


In [39]:
from tensorflow.keras.models import load_model
model=load_model('/content/Potato_model1.h5',compile=False)